In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [ ]:
# count the number of images in the respective classes 0 - Brain Tumor and 1 - Healthy
ROOT_DIR1 = "dataset"
number_of_images = {}
total_training_images=0

# os.listdir gives all the items present in the directory
# os.path.join joins the parameter so that we can go to next directory

for dir in os.listdir(ROOT_DIR1):
  number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR1,dir)))
  total_training_images+=number_of_images[dir]

print("Number of images in each folder of Training set:",number_of_images)
print("Total number of images in training set",total_training_images)

**We will split the data such that 70% for Train Data , 15% for Validation and rest 15% for Testing**

In [ ]:
# Function which will create a folder and spilts the images acoordingly in that percentage.

def dataFolder(path , split):
  ROOT = "directory\\"

  if not os.path.exists(ROOT+path):
    os.mkdir(ROOT+path)

    for dir in os.listdir(ROOT_DIR1):
      os.makedirs(ROOT+path+'\\'+ dir)

      for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR1,dir)) , 
                                  size = (math.floor(split*number_of_images[dir])-5) , 
                                  replace=False):
        O = os.path.join(ROOT_DIR1,dir,img) # path and not the image
        D = os.path.join(ROOT+path,dir)
        shutil.copy(O,D)
    print(f"{path} Folder created")
        # os.remove(O)      # The image which wil copied in new directory(Train,Test,Validation) will be deleted from the original directory(here BrainTumorData)
  else:
    print(f"{path} Folder exists")

In [ ]:
# Making separate folders with certain number of images
dataFolder("Train",0.70)
dataFolder("Test",0.15)
dataFolder("Validation",0.15)

### **Preprocessing or Preparing our data using Data Generator**

In [ ]:
import keras
import tensorflow
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input

In [ ]:
# Creating a function for processing train images

def preprocessingImages1(path):
  """
  input : Path
  output : Generated Images
  """
  image_data = ImageDataGenerator(zoom_range= 0.2 , shear_range= 0.2 ,  preprocessing_function = preprocess_input , horizontal_flip= True) # data augmentation
  image = image_data.flow_from_directory(directory = path, target_size = (224,224) , batch_size = 32 , class_mode= 'binary')  
  # taget_size should be same as input size i.e 224,224 and channel will be taken automatically
  # class_mode is binary because of binary classification

  return image



In [ ]:
path = "director\\Train"
train_data = preprocessingImages1(path)


In [ ]:
# Creating a function for processing testing and validation images

def preprocessingImages2(path):
  """
  input : Path
  output : Generated Images
  """
  image_data = ImageDataGenerator(preprocessing_function= preprocess_input)
  image = image_data.flow_from_directory(directory = path, target_size = (224,224) , batch_size = 32 , class_mode= 'binary')  

  return image



In [ ]:
path = "director\\Test"
test_data = preprocessingImages2(path)

In [ ]:
path = "director\\Validation"
val_data = preprocessingImages2(path)

# **Model Training**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense
from keras.models import Model, load_model
from keras.applications.mobilenet import MobileNet
import keras

In [ ]:
base_model = MobileNet(input_shape=(224,224,3), include_top= False)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
X = Flatten()(base_model.output)
X = Dense(units= 1 , activation= 'sigmoid')(X)

model = Model(base_model.input, X)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer= 'rmsprop' , loss= keras.losses.binary_crossentropy, metrics= ['accuracy'])

In [ ]:
# Call Back
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Model checkpoint
mcp = ModelCheckpoint(filepath= "bestmodelmobilenet.h5", monitor= 'val_accuracy' , verbose= 1 , save_best_only= True )


# Early Stopping
es = EarlyStopping(monitor= 'val_accuracy' , min_delta= 0.01 , patience= 3 , verbose= 1)

cb = [mcp,es]



In [ ]:
hist = model.fit_generator(train_data , 
                           steps_per_epoch= 8 , 
                           epochs= 300 , 
                           validation_data= val_data , 
                           validation_steps= 6 , 
                           callbacks= cb )

In [ ]:
# Load the best fit model
model = load_model("bestmodelmobilenet.h5")

In [ ]:
acc = model.evaluate_generator(test_data)[1]

print(f"Our Model accuracy is {acc*100} %")

In [ ]:
h = hist.history
h.keys()

In [ ]:
# Plotting for accuracy
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c='red')
plt.title("Accuracy Vs Val_Accuracy")
plt.legend(["accuracy", "val_accuracy"], loc ="lower right")
plt.show()

In [ ]:
# Plotting for loss
plt.plot(h['loss'])
plt.plot(h['val_loss'], c='red')
plt.title("Loss Vs Val_Loss")
plt.legend(["loss", "val_loss"], loc ="upper right")
plt.show()

In [ ]:
# !pip install Keras-Preprocessing
from keras.preprocessing import image
from keras_preprocessing.image import load_img, img_to_array

In [ ]:
# giving that image path which we haven't preprocessed i.e from Dataset folder

path = "dataset\\healthy\\31 no.jpg"

img = load_img(path,target_size= (224,224))

i = img_to_array(img)/255
input_arr = np.array([i])
input_arr.shape


pred = (model.predict(input_arr) > 0.5).astype("int32")[0][0]
print("Prediction",pred)

if pred==0:
  print("The MRI image is of healthy brain")
else:
  print("The MRI image is of Brain tumor")


# to display the image
plt.imshow(input_arr[0])
plt.title("Input Image")
plt.show()